# Artificial Intelligence

## Import Libraries

In [1]:
from ai.DataLoader import DataLoader
from ai.TrashClassifier import TrashClassifier
from env import *

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
train_dloader = DataLoader(TRASH_TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(TRASH_VALID_DATA_PATH, TRASH_CAT)

Number of data batch: 503
Number of batch: 51
Number of data batch: 62
Number of batch: 7


In [3]:
model = TrashClassifier()
model = model.cuda()

In [4]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=ETA)

In [ ]:
for e in range(EPOCHS):
    
    # loss and accuracy on train dataset.
    train_loss = 0.0
    train_acc = 0.0
    
    # loss and accuracy on valid dataset.
    val_loss = 0.0
    val_acc = 0.0
    
    # train step
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.tensor(x_batch).float().cuda()
        y_batch = torch.tensor(y_batch).long().cuda()
        
        # forward propagation
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        train_loss += loss.item()
        
        # compute train accuracy
        with torch.no_grad():
            ps = torch.exp(logps)
            val_k, ind_k = ps.topk(1, dim=1)
            equal = ind_k == y_batch.view(*ind_k.shape)
            train_acc += torch.mean(equal.type(torch.FloatTensor)).item()
            
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # validation step
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in valid_dloader.next_batch():
            x_batch = torch.tensor(x_batch).float().cuda()
            y_batch = torch.tensor(y_batch).long().cuda()
            
            # compute validation loss
            logps = model(x_batch)
            loss = criterion(logps, y_batch)
            val_loss += loss.item()
            
            # compute validation accuracy
            ps = torch.exp(logps)
            value_k, index_k = ps.topk(1, dim=1)
            equal = index_k == y_batch.view(*index_k.size())
            val_acc += torch.mean(equal.type(torch.FloatTensor)).item()
        
    train_loss /= len(train_dloader)
    train_acc /= len(train_dloader)
    val_loss /= len(valid_dloader)
    val_acc /= len(valid_dloader)
    
    print(f"Epochs {e+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.8f}")
    print(f"Train acc: {train_acc:.8f}")
    print(f"Valid loss: {val_loss:.8f}")
    print(f"Valid acc: {val_acc:.8f}")
    
    model.train()